# **Spectral Index Evaluation of Land Use Compliance with the PBOT in La Ceja**
This project aims to analyze and quantify the urban expansion in La Ceja, a municipality located in Eastern Antioquia (Colombia), between the years 2018 and 2025. Using Sentinel-2 multispectral satellite imagery, we calculate spectral indices such as NDVI (Normalized Difference Vegetation Index) and NDBI (Normalized Difference Built-up Index) to detect changes in vegetation cover and built-up areas.

These spatial transformations are then compared with the urban growth zones defined in the Municipal Land Use Plan (PBOT) established by Agreement 001 of 2018. The overlay between remote sensing analysis and official planning boundaries allows us to assess whether recent urban development has occurred within legally designated areas or has exceeded regulated expansion zones.

This approach provides technical evidence to support sustainable territorial management, monitor compliance with planning regulations, and inform future decision-making by local authorities. The project also explores the effectiveness of remote sensing as a tool for land-use monitoring in rapidly growing intermediate municipalities.


## CORTE DE LAS IMAGÉNES 

In [2]:
import os
import rasterio
from rasterio.mask import mask
from pyproj import Transformer

# === CONFIGURACIÓN ===
folder_b4 = "carpeta_b4"
folder_b8 = "carpeta_b8"
folder_b11 = "carpeta_b11"

output_b4 = "recortadas_b4"
output_b8 = "recortadas_b8"
output_b11 = "recortadas_b11"

# Crear carpetas de salida si no existen
os.makedirs(output_b4, exist_ok=True)
os.makedirs(output_b8, exist_ok=True)
os.makedirs(output_b11, exist_ok=True)

# === COORDENADAS Y TRANSFORMACIÓN ===
coords_wgs84 = [
    (-75.458237, 6.015422),  # inf izq (lon, lat)
    (-75.407993, 6.015422),  # inf der
    (-75.407993, 6.040884),  # sup der
    (-75.458237, 6.040884),  # sup izq
    (-75.458237, 6.015422)   # cierre
]

transformer = Transformer.from_crs("EPSG:4326", "EPSG:32618", always_xy=True)
coords_utm = [transformer.transform(lon, lat) for lon, lat in coords_wgs84]

polygon = {
    "type": "Polygon",
    "coordinates": [coords_utm]
}

# === FUNCIONES ===
def recortar_guardar(input_path, output_path, polygon):
    with rasterio.open(input_path) as src:
        out_image, out_transform = mask(src, [polygon], crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)

# === PROCESAMIENTO ===
for archivo_b4 in os.listdir(folder_b4):
    if archivo_b4.endswith(".TIF"):
        ruta_b4 = os.path.join(folder_b4, archivo_b4)

        # Suponemos mismo nombre base, solo cambia B4 -> B8/B11
        archivo_b8 = archivo_b4.replace("B4", "B8")
        archivo_b11 = archivo_b4.replace("B4", "B11")

        ruta_b8 = os.path.join(folder_b8, archivo_b8)
        ruta_b11 = os.path.join(folder_b11, archivo_b11)

        if os.path.exists(ruta_b8) and os.path.exists(ruta_b11):
            print(f"✅ Procesando: {archivo_b4}, {archivo_b8}, {archivo_b11}")

            salida_b4 = os.path.join(output_b4, f"rec_{archivo_b4}")
            salida_b8 = os.path.join(output_b8, f"rec_{archivo_b8}")
            salida_b11 = os.path.join(output_b11, f"rec_{archivo_b11}")

            recortar_guardar(ruta_b4, salida_b4, polygon)
            recortar_guardar(ruta_b8, salida_b8, polygon)
            recortar_guardar(ruta_b11, salida_b11, polygon)
        else:
            print(f"❌ Faltan archivos para: {archivo_b4}")


FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'carpeta_b4'